In [1]:
import pandas as pd
import json

# 1. Initial Compromise - Frontend Service

## Get Frontend Data

### Provenance Data

In [2]:
# provenance data dfs
artifacts_df = pd.read_csv("../provenance_data/app_artifact_vertices.csv")
processes_df = pd.read_csv("../provenance_data/app_process_vertices.csv")
edges_df = pd.read_csv("../provenance_data/app_edges.csv")

In [3]:
artifacts_df.head()

,Unnamed: 0,id,type,path,subtype,permissions,epoch,source,version,local address,...,protocol,remote address,local port,net namespace,memory address,size,tgid,read fd,write fd,svc
0,48,a41db92088a31a823a23ed48145dd661,Artifact,NaN,network socket,NaN,0.0,syscall,0,0.0.0.0,...,tcp,172.31.30.227,6379.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,redis-cart
1,49,a2f50c6b069415c603efdac935d00de4,Artifact,NaN,network socket,NaN,0.0,syscall,0,0.0.0.0,...,tcp,172.31.30.227,6379.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,redis-cart
2,96,be892937315bea0644510e695d37ac30,Artifact,NaN,network socket,NaN,0.0,syscall,0,127.0.0.1,...,tcp,0.0.0.0,46636.0,4.026532e+09,NaN,NaN,NaN,NaN,NaN,frontend
3,97,5a7a49b0a1c2096d4e6e896d35f9194d,Artifact,NaN,network socket,NaN,0.0,syscall,0,0000:0000:0000:0000:0000:0000:0000:0000,...,tcp,0000:0000:0000:0000:0000:FFFF:7F00:0001,8080.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,frontend
4,98,80a3b6046c1aa2dda29b76dc32d1e4fa,Artifact,NaN,network socket,NaN,0.0,syscall,0,127.0.0.1,...,tcp,0.0.0.0,46642.0,4.026532e+09,NaN,NaN,NaN,NaN,NaN,frontend


In [4]:
# Filter frontend service
frontend_artifacts_df = artifacts_df[artifacts_df['svc'] == "frontend"]
frontend_processes_df = processes_df[processes_df['svc'] == "frontend"]

In [5]:
# Create sets of unique IDs
frontend_ids = set(frontend_artifacts_df['id']).union(set(frontend_processes_df['id']))

# Filter edges_df based on whether 'from' or 'to' vertices are in the sets of IDs
frontend_edges = edges_df[edges_df['from'].isin(frontend_ids) | edges_df['to'].isin(frontend_ids) | edges_df['from'].isin(frontend_ids) | edges_df['to'].isin(frontend_ids)]

frontend_edges

,Unnamed: 0,from,to,type,event id,flags,source,time,operation,protection,advice
4,915,0ef366a12c22d68dcc35e410ab69719e,413b8704b3209061ce63aaa13a274964,WasTriggeredBy,1307,NaN,syscall,1.702120e+09,execve,NaN,NaN
5,916,0ef366a12c22d68dcc35e410ab69719e,463300ec78aa5cf048742792ee5e5ef4,Used,1307,NaN,syscall,1.702120e+09,load,NaN,NaN
6,917,0ef366a12c22d68dcc35e410ab69719e,ff24d51b85635ab8e2118892169b5c1d,Used,1308,O_RDONLY,syscall,1.702120e+09,open,NaN,NaN
7,918,0ef366a12c22d68dcc35e410ab69719e,ff24d51b85635ab8e2118892169b5c1d,Used,1309,NaN,syscall,1.702120e+09,close,NaN,NaN
8,919,9154e1fe447110ceac41673360bce4a7,0ef366a12c22d68dcc35e410ab69719e,WasGeneratedBy,1310,NaN,syscall,1.702120e+09,mmap (write),3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2376,36652,c9b24b4a9b63a52584108840c8d05916,3755bde2467599dfa823ec65cb26788d,Used,53167,NaN,syscall,1.702120e+09,accept,NaN,NaN
2380,36658,c95375d1d0eea3b92c3d14a7e08b1861,afcfa21812d2f719e81ab88ad8db4765,Used,53183,NaN,syscall,1.702120e+09,accept,NaN,NaN
2381,36659,afcfa21812d2f719e81ab88ad8db4765,f7463923246daf094a0d944c90db0146,WasGeneratedBy,53186,NaN,syscall,1.702120e+09,close,NaN,NaN
2445,37591,c95375d1d0eea3b92c3d14a7e08b1861,44593fa4d9a1562de7a6c9d5cc888d70,Used,54418,NaN,syscall,1.702120e+09,accept,NaN,NaN


### CVE Data

In [26]:
# from vulnerabilities-scanner import trivy_cve_parser

def get_cve_df_for_service(service_name):
    """ Extract relevant CVE information from Trivy JSON output" - to a specific service given as input"""
    # Read Trivy JSON file
    vuln_json_file = open('../../vulnerability-handler/Trivy/trivy_vuln_scan.json') # 2023-12-04
    trivy_dict = json.loads(vuln_json_file.read())
    # Initialize an empty DataFrame for CVE data
    df_vuln_svc = pd.DataFrame(columns=['Pod-ID', 'Software', 'Version', 'CVE-ID', 'CVE-Severity', 'CVE-Title', 'CVSS'])
    # Extract relevant information from Trivy JSON
    findings_list = trivy_dict['Findings']
    for pod_vulns_dict in findings_list:
        if 'Namespace' in pod_vulns_dict.keys() and pod_vulns_dict['Namespace'] == 'attack-graphs':
            if 'Results' in pod_vulns_dict:
                for res_dict in pod_vulns_dict['Results']:
                    if 'Vulnerabilities' in res_dict:
                        for vuln_dict in res_dict['Vulnerabilities']:
                            # if service_name in pod_vulns_dict['Name']:
                            # Check if service name matches any in the list
                            if service_name in pod_vulns_dict['Name']:
                                new_vuln_row = {
                                    'Pod-ID': pod_vulns_dict['Name'],
                                    'Software': vuln_dict['PkgName'],
                                    'Version': vuln_dict['InstalledVersion'],
                                    'CVE-ID': vuln_dict["VulnerabilityID"],
                                    'CVE-Severity': vuln_dict["Severity"],
                                    'CVE-Title': vuln_dict["Title"] if 'Title' in vuln_dict else None,
                                    'CVSS': vuln_dict["CVSS"] if 'CVSS' in vuln_dict else None
                                }
                                df_vuln_svc = pd.concat([df_vuln_svc, pd.DataFrame([new_vuln_row])])
    return df_vuln_svc


In [27]:
df_vuln_frontend2 = get_cve_df_for_service("frontend")
df_vuln_frontend2

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS
0,frontend-6,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-6,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-4,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-4,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-3,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-3,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-5,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-5,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-11,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-11,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...


# CVE Data for Services

## Cart & Redis Services

In [28]:
df_vuln_cart = get_cve_df_for_service("cart")
df_vuln_cart

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS
0,cartservice-1,golang.org/x/net,v0.0.0-20220906165146-f3363e06e74c,CVE-2022-41717,MEDIUM,An attacker can cause excessive memory growth ...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,cartservice-1,golang.org/x/net,v0.0.0-20220906165146-f3363e06e74c,CVE-2022-41721,UNKNOWN,None,None


In [29]:
df_vuln_redis = get_cve_df_for_service("redis")
df_vuln_redis

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS


## Recommendation Service

In [30]:
df_vuln_recommend = get_cve_df_for_service("recommendation")

In [31]:
df_vuln_recommend

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS
0,recommendationservice-7,apt,2.2.4,CVE-2011-3374,LOW,"It was found that apt-key in apt, all versions...",{'nvd': {'V2Vector': 'AV:N/AC:M/Au:N/C:N/I:P/A...
0,recommendationservice-7,bash,5.1-2+deb11u1,CVE-2022-3715,HIGH,bash: a heap-buffer-overflow in valid_paramete...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:L/AC:L/PR:L/...
0,recommendationservice-7,bsdutils,2.36.1-8+deb11u1,CVE-2022-0563,LOW,util-linux: partial disclosure of arbitrary fi...,{'nvd': {'V2Vector': 'AV:L/AC:M/Au:N/C:P/I:N/A...
0,recommendationservice-7,coreutils,8.32-4,CVE-2016-2781,LOW,coreutils: Non-privileged session can escape t...,{'nvd': {'V2Vector': 'AV:L/AC:L/Au:N/C:N/I:P/A...
0,recommendationservice-7,coreutils,8.32-4,CVE-2017-18018,LOW,coreutils: race condition vulnerability in cho...,{'nvd': {'V2Vector': 'AV:L/AC:M/Au:N/C:N/I:P/A...
...,...,...,...,...,...,...,...
0,recommendationservice-10,tar,1.34+dfsg-1,CVE-2005-2541,LOW,tar: does not properly warn the user when extr...,{'nvd': {'V2Vector': 'AV:N/AC:L/Au:N/C:C/I:C/A...
0,recommendationservice-10,util-linux,2.36.1-8+deb11u1,CVE-2022-0563,LOW,util-linux: partial disclosure of arbitrary fi...,{'nvd': {'V2Vector': 'AV:L/AC:M/Au:N/C:P/I:N/A...
0,recommendationservice-10,setuptools,63.2.0,CVE-2022-40897,HIGH,pypa-setuptools: Regular Expression Denial of ...,{'ghsa': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N...
0,recommendationservice-10,golang.org/x/net,v0.0.0-20220906165146-f3363e06e74c,CVE-2022-41717,MEDIUM,An attacker can cause excessive memory growth ...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...


In [32]:
df_vuln_recommend.drop_duplicates(subset=df_vuln_recommend.columns.difference(['Pod-ID','CVSS']))

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS
0,recommendationservice-7,apt,2.2.4,CVE-2011-3374,LOW,"It was found that apt-key in apt, all versions...",{'nvd': {'V2Vector': 'AV:N/AC:M/Au:N/C:N/I:P/A...
0,recommendationservice-7,bash,5.1-2+deb11u1,CVE-2022-3715,HIGH,bash: a heap-buffer-overflow in valid_paramete...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:L/AC:L/PR:L/...
0,recommendationservice-7,bsdutils,2.36.1-8+deb11u1,CVE-2022-0563,LOW,util-linux: partial disclosure of arbitrary fi...,{'nvd': {'V2Vector': 'AV:L/AC:M/Au:N/C:P/I:N/A...
0,recommendationservice-7,coreutils,8.32-4,CVE-2016-2781,LOW,coreutils: Non-privileged session can escape t...,{'nvd': {'V2Vector': 'AV:L/AC:L/Au:N/C:N/I:P/A...
0,recommendationservice-7,coreutils,8.32-4,CVE-2017-18018,LOW,coreutils: race condition vulnerability in cho...,{'nvd': {'V2Vector': 'AV:L/AC:M/Au:N/C:N/I:P/A...
...,...,...,...,...,...,...,...
0,recommendationservice-7,tar,1.34+dfsg-1,CVE-2005-2541,LOW,tar: does not properly warn the user when extr...,{'nvd': {'V2Vector': 'AV:N/AC:L/Au:N/C:C/I:C/A...
0,recommendationservice-7,util-linux,2.36.1-8+deb11u1,CVE-2022-0563,LOW,util-linux: partial disclosure of arbitrary fi...,{'nvd': {'V2Vector': 'AV:L/AC:M/Au:N/C:P/I:N/A...
0,recommendationservice-7,setuptools,63.2.0,CVE-2022-40897,HIGH,pypa-setuptools: Regular Expression Denial of ...,{'ghsa': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N...
0,recommendationservice-7,golang.org/x/net,v0.0.0-20220906165146-f3363e06e74c,CVE-2022-41717,MEDIUM,An attacker can cause excessive memory growth ...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...


## Frontend

In [33]:
df_vuln_frontend3 = get_cve_df_for_service("frontend")
df_vuln_frontend3

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS
0,frontend-6,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-6,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-4,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-4,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-3,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-3,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-5,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-5,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-11,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-11,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...


In [34]:
df_vuln_frontend3.drop_duplicates(subset=df_vuln_frontend3.columns.difference(['Pod-ID','CVSS']))

,Pod-ID,Software,Version,CVE-ID,CVE-Severity,CVE-Title,CVSS
0,frontend-6,libcrypto3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
0,frontend-6,libssl3,3.0.7-r0,CVE-2022-3996,HIGH,openssl: double locking leads to denial of ser...,{'nvd': {'V3Vector': 'CVSS:3.1/AV:N/AC:L/PR:N/...
